In [1]:
'''importing required lib'''
import numpy as np # to deal with numbers and array
import pandas as pd #for handling dataframes
from sklearn.utils import shuffle #use to shuffle dataframe

''' lib required for creating a model  i.e text data into array of matrix'''
from sklearn.feature_extraction.text import CountVectorizer  ## BOW Model
from sklearn.feature_extraction.text import TfidfVectorizer  ## TFIDF Model
# for converting text data into array 

''' lib required for cleaning of data'''
from nltk.corpus import stopwords #containing stopwords
from nltk.stem.porter import PorterStemmer # use to fing a core meaning of words 
from nltk.stem.wordnet import WordNetLemmatizer
import re #used for regular expression operation
import os
import nltk

'''Algorithms used
naivebayes>>>MultinomialNB ,BernoulliNB
linear_model>>>>>Logistic regression ,SGDClassifier, 
SVM>>>>SVC ,LinearSVC ,NuSVC
RandomForest
'''

'Algorithms used\nnaivebayes>>>MultinomialNB ,BernoulliNB\nlinear_model>>>>>Logistic regression ,SGDClassifier, \nSVM>>>>SVC ,LinearSVC ,NuSVC\nRandomForest\n'

In [2]:
dataframe=pd.read_csv('crisis2.csv',encoding = 'unicode_escape')

In [3]:
del dataframe['Unnamed: 0']

In [4]:
dataframe.head() #showing first five values

,label,tweet_id,tweet_text
0,other_useful_information,'592326564110585856',RT @divyaconnects: Reached #Kathmandu finally!...
1,missing_trapped_or_found_people,'592616512642420737',fears for Foreigners missing in Nepal earthqua...
2,affectedPeople_or_injured_or_dead_people_or_De...,'592686635520827393',"RT @ParisBurned: 3,700 people dead is absolute..."
3,donation_needs_or_offers_or_volunteering_services,'593301431366635520',Earthquake in Nepal - Please help Kapil #crowd...
4,other_useful_information,'592590231519555584',NepalÂs Slowing Economy Set for Freefall With...


In [5]:
dataframe.shape # describing the size of dataframe

(17866, 3)

In [6]:
dataframe.columns #information about the attributes

Index(['label', 'tweet_id', 'tweet_text'], dtype='object')

In [7]:
dataframe.describe() # describe the dataframe internal information

,label,tweet_id,tweet_text
count,17866,17866,17866
unique,9,16297,16297
top,other_useful_information,'384668377278472192',Watch #NDMA Chairman Major General Syed Aleem'...
freq,6231,2,2


In [8]:
dataframe.isnull().sum() #determining whether there is a empty value is present or not

label         0
tweet_id      0
tweet_text    0
dtype: int64

In [9]:
dataframe['label'].unique()

array(['other_useful_information', 'missing_trapped_or_found_people',
       'affectedPeople_or_injured_or_dead_people_or_DeathReports',
       'donation_needs_or_offers_or_volunteering_services',
       'sympathy_and_emotional_support',
       'prevention_or_caution_and_advice',
       'infrastructure_and_utilities_damage',
       'disease_transmission_or_disease_signs_or_symptoms', 'treatment'],
      dtype=object)

In [10]:
def same_class(x):
    if x == "deaths_reports" or x=="injured_or_dead_people" or x=="affected_people":
        return "affectedPeople_or_injured_or_dead_people_or_DeathReports"
    elif x=="disease_transmission" or x=="disease_signs_or_symptoms":
        return "disease_transmission_or_disease_signs_or_symptoms"
    elif x=="displaced_people_and_evacuations" or x=="missing_trapped_or_found_people":
        return 'missing_trapped_or_found_people'
    elif x=="caution_and_advice" or x=="prevention":
        return "prevention_or_caution_and_advice"
    else:
        return x

In [11]:

dataframe["label"]=dataframe["label"].map(same_class)      

In [12]:
print(dataframe['label'].value_counts()) # number of events with there count

other_useful_information                                    6231
affectedPeople_or_injured_or_dead_people_or_DeathReports    3051
donation_needs_or_offers_or_volunteering_services           2288
sympathy_and_emotional_support                              1875
infrastructure_and_utilities_damage                         1336
prevention_or_caution_and_advice                            1282
missing_trapped_or_found_people                              751
disease_transmission_or_disease_signs_or_symptoms            682
treatment                                                    370
Name: label, dtype: int64


In [13]:
#dataframe=dataframe.drop(dataframe[(dataframe['label']=="other_useful_information")].index)

In [14]:
#dataframe[dataframe['label']=="not_related_or_irrelevant"]

In [15]:
#dataframe.to_csv('crisis3.csv')

In [16]:
label1={'other_useful_information':0, 'missing_trapped_or_found_people':1,
       'affectedPeople_or_injured_or_dead_people_or_DeathReports':2,
       'donation_needs_or_offers_or_volunteering_services':3,
       'sympathy_and_emotional_support':4, 'not_related_or_irrelevant':5,
       'prevention_or_caution_and_advice':6,
       'infrastructure_and_utilities_damage':7,
       'disease_transmission_or_disease_signs_or_symptoms':8, 'treatment':9}
def label(x):
    return label1[x];

In [17]:
dataframe["numeric_label"]=dataframe["label"].map(label)

In [18]:
dataframe['label'].value_counts() # number of events with there count 

other_useful_information                                    6231
affectedPeople_or_injured_or_dead_people_or_DeathReports    3051
donation_needs_or_offers_or_volunteering_services           2288
sympathy_and_emotional_support                              1875
infrastructure_and_utilities_damage                         1336
prevention_or_caution_and_advice                            1282
missing_trapped_or_found_people                              751
disease_transmission_or_disease_signs_or_symptoms            682
treatment                                                    370
Name: label, dtype: int64

In [19]:
dataframe['numeric_label'].value_counts() # number of events with there count 

0    6231
2    3051
3    2288
4    1875
7    1336
6    1282
1     751
8     682
9     370
Name: numeric_label, dtype: int64

In [22]:
dataframe.shape

(17866, 4)

In [20]:
print(dataframe['tweet_text'][5])
print("event extracted>>>")
print(dataframe['label'][5])
'''tweet with corresponding label'''

RT @cuteinsan: In this horrible situation in nepal our God  @Gurmeetramrahim  can help pita ji they really need your simpathy #MSGHelpEarthÂ
event extracted>>>
sympathy_and_emotional_support


'tweet with corresponding label'

In [21]:
dataframe['tweet_id'].nunique() #number of unique tweet ID

16297

In [23]:
stop_words = set(stopwords.words('english'))
print(stop_words)

{'ve', 'herself', "mightn't", 'few', 'o', 'about', 'as', 'below', 'mustn', 'wasn', 'won', 'yourself', "it's", 'theirs', 'both', 'that', 'only', 'a', 'is', 'other', "you've", 'him', 'having', 'they', "doesn't", 'each', 'what', 'don', "you'd", 'my', "needn't", 'ours', 't', 'by', 'while', 'y', "mustn't", 'why', 'whom', 'over', 'just', 'after', 'up', 'there', 'll', 'it', 'now', "you're", 'did', 'ma', 'do', 'itself', 'does', 'above', 'an', 'be', 'shan', 're', 'once', 'own', 'was', 'down', 'this', 'here', 'didn', 'yourselves', 'between', 'and', 'you', 'these', 'm', 'such', 'no', 'than', 'themselves', 'our', 's', "weren't", "should've", 'nor', 'yours', 'himself', 'has', 'or', "isn't", 'should', 'but', 'off', 'ain', 'any', 'her', 'am', "won't", "didn't", 'hers', 'me', 'its', 'doing', 'she', "hasn't", 'if', 'at', 'before', 'to', "don't", 'how', 'not', 'were', 'under', 'your', "you'll", 'couldn', 'through', 'in', 'mightn', "haven't", 'the', "aren't", 'shouldn', 'd', 'have', "that'll", 'i', 'duri

In [24]:
new_words = ["RT",'rt'] #creating a list of extra stop words
stop = stop_words.union(new_words) #adding words into corpus

In [25]:
sno = nltk.stem.SnowballStemmer('english') #initialising the snowball stemmer

In [26]:
def cleanhtml(sentence):
    '''This function removes all the html tags in the given sentence'''
    cleanr = re.compile('<.*?>')    ## find the index of the html tags
    cleantext = re.sub(cleanr, ' ', sentence)  ## Substitute <space> in place of any html tag
    return cleantext

In [27]:
def cleanpunc(sentence):
    '''This function cleans all the punctuation or special characters from a given sentence'''
    cleaned = re.sub(r'[?|@|!|^|%|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return  cleaned

In [28]:
def preprocessing(series):
    '''The function takes a Pandas Series object containing text in all the cells
       And performs following Preprocessing steps on each cell:
       1. Clean text from html tags
       2. Clean text from punctuations and special characters
       3. Retain only non-numeric Latin characters with lenght > 2
       4. Remove stopwords from the sentence
       5. Apply stemming to all the words in the sentence
       
       Return values:
       1. final_string - List of cleaned sentences
       2. list_of_sent - List of lists which can be used as input to the W2V model'''
    
    i = 0
    str1=" "
    final_string = []    ## This list will contain cleaned sentences
    list_of_sent = []    ## This is a list of lists used as input to the W2V model at a later stage
    
    ## Creating below lists for future use
    all_positive_words=[] # store words from +ve reviews here
    all_negative_words=[] # store words from -ve reviews here
    
    
    for sent in series.values:
        ## 
        filtered_sent = []
        
        sent = cleanhtml(sent)    ## Clean the HTML tags
        sent = cleanpunc(sent)    ## Clean the punctuations and special characters
        ## Sentences are cleaned and words are handled individually
        for cleaned_words in sent.split():
            ## Only consider non-numeric words with length at least 3
            if((cleaned_words.isalpha()) & (len(cleaned_words) > 2)):
                ## Only consider words which are not stopwords and convert them to lowet case
                if(cleaned_words.lower() not in stop):
                    ## Apply snowball stemmer and add them to the filtered_sent list
                    s = (sno.stem(cleaned_words.lower()))#.encode('utf-8')
                    filtered_sent.append(s)    ## This contains all the cleaned words for a sentence
                    '''
                    if (final['Score'].values)[i] == 'positive':
                        all_positive_words.append(s) #list of all words used to describe positive reviews
                    if(final['Score'].values)[i] == 'negative':
                        all_negative_words.append(s) #list of all words used to describe negative reviews
                    '''
        ## Below list is a list of lists used as input to W2V model later
        list_of_sent.append(filtered_sent)
        ## Join back all the words belonging to the same sentence
        str1 = " ".join(filtered_sent)
        ## Finally add the cleaned sentence in the below list
        final_string.append(str1)
        #print(i)
        i += 1
    return final_string, list_of_sent

In [29]:
final_string, list_of_sent = preprocessing(dataframe['tweet_text'])

In [30]:
len(final_string)

17866

In [31]:
list_of_sent[2]

['peopl',
 'dead',
 'absolut',
 'devast',
 'countri',
 'size',
 'oklahoma',
 'map',
 'nepal']

In [32]:
import gensim
w2v_model=gensim.models.Word2Vec(list_of_sent,min_count=5,size=50, workers=4)

In [33]:
words = list(w2v_model.wv.vocab)
print(len(words))

2972


In [34]:
w2v_model.wv.most_similar('water')

[('medic', 0.9899135828018188),
 ('work', 0.9794239401817322),
 ('provid', 0.9789196848869324),
 ('respons', 0.9744434952735901),
 ('med', 0.9688825607299805),
 ('sanit', 0.9643027782440186),
 ('team', 0.9606571197509766),
 ('govt', 0.9601358771324158),
 ('food', 0.9586820602416992),
 ('contact', 0.9573441743850708)]

In [79]:
def calc_tfidf_avg_w2v(list_of_sent, w2v_model, tf_idf, tfidf_feat):
    '''This function takes in 4 parameters as follows:
       1. list_of_sent - This is the list of sentences/reviews for which sentence vetors are to be constructed
       2. w2v_model - This is the Word2Vec model which is trained on the working corpus - contains the word vectors
       3. tf_idf - This is the TF-IDF model built using the same reviews/sentences - it is the TF-IDF sparse matrix
       4. tfidf_feat - This is the feature vector constructed from the TF-IDF model
       
       Return Value:
       tfidf_sent_vectors - This is a list of sentence/review vectors constructed by using tfidf weighted average on the word vectors
    '''
    
    ## Initialize an empty list
    tfidf_sent_vectors = []
    row = 0
    ## Consider one sentence/review at a time
    for sent in list_of_sent:
        ## Initialize sentence vector to 0
        sent_vec = np.zeros(50)
        ## Initialize weighted sum to 0
        weighted_sum = 0
        ## Consider the words one by one
        for word in sent:
            try:
                ## Calculate the word vector using the W2V model
                vec = w2v_model.wv[word]
                ## Calculate tfidf value of the word in that review using tfidf model
                tfidf = tf_idf[row, tfidf_feat.index(word)]
                ## Add the product of tfidf*word_vec to the sentence vector (This is the numerator)
                sent_vec += vec*tfidf
                ## Sum all the tfidf values (This is the denominator)
                weighted_sum += tfidf
            except:
                pass
        #print(row, weighted_sum)
        
        ## Divide the numerator by the denominator to get the sentence vector
        sent_vec /= weighted_sum
        ## Add the sentence vector in the final list
        tfidf_sent_vectors.append(sent_vec)
        row += 1
    ## return the list of all the sentence vectors
    return tfidf_sent_vectors

In [80]:
## Preparing the data to call the function

## Make sure "list_of_sent" is imported from the pickle file
tf_idf_vect = TfidfVectorizer()
tf_idf = tf_idf_vect.fit_transform(final_string)
tfidf_feat = tf_idf_vect.get_feature_names()
w2v_model=gensim.models.Word2Vec(list_of_sent,min_count=5,size=50, workers=4)

In [81]:
tfidf_sent_vectors = calc_tfidf_avg_w2v(list_of_sent, w2v_model, tf_idf, tfidf_feat)

C:\Users\shailesh\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in true_divide


In [82]:
len(tfidf_sent_vectors)

17866

In [83]:
tfidf_vec_array=np.array(tfidf_sent_vectors)
tfidf_vec_array.shape

(17866, 50)

In [84]:
## Remove all the rows from the array which contain NaN values
tfidf_vec_array1 = tfidf_vec_array[~np.isnan(tfidf_vec_array).any(axis = 1)]
tfidf_vec_array1.shape

(17861, 50)

In [85]:
where_are_nans=np.isnan(tfidf_vec_array)

In [86]:
tfidf_vec_array[where_are_nans]=0

In [87]:
y=dataframe['numeric_label'].values

In [88]:
from sklearn.model_selection import train_test_split

In [89]:
X_train, X_test, y_train, y_test = train_test_split(tfidf_vec_array, y, test_size = 0.1, random_state = 123)

In [90]:
from sklearn import metrics

In [91]:
from sklearn.svm import LinearSVC
linear_svc_model = LinearSVC() # note perform changes in attribute to get bettter accuracy
linear_svc_model.fit(X_train,y_train)#passing data into model for training
y_pred=linear_svc_model.predict(X_test)#after training we pass rest 20%of data for testing purpose
acc=metrics.accuracy_score(y_test,y_pred) 
#here we are passing actual result and predicted result to find out accuracy
print('acuuracy>>>>',acc*100)

acuuracy>>>> 66.31225517627308


In [92]:
final_string

['reach kathmandu final lot indian strand airport nepalquak bbchindi',
 'fear foreign miss nepal earthquak death toll',
 'peopl dead absolut devast countri size oklahoma map nepal',
 'earthquak nepal pleas help kapil crowdfund fundrazr support retweet via mechanismofwar',
 'slow economi set freefal without help kszimvhgqf via busi',
 'horribl situat nepal god gurmeetramrahim help pita realli need simpathi msghelpearthâ',
 'video report kathmandu overwhelm rubbl earthquak dead plus injur nepalearthquak',
 'peopl nepal need help find support relief effort',
 'nepal earthquak deathtol rise nepalearthquak prayfornep earthquak',
 'aid lorri deliv nepalearthquak relief hand landslid prevent vehicl access',
 'food crisi immin million ppl kathmandu long would last without food market water tap',
 'canadian phone compani waiv fee call nepal globe mail',
 'word ramdev nepal help peopl that qualiti work peopl talk ill',
 'dont believ earthquak happen natur mother natur theyr provok avoid',
 'deat

In [146]:
x=[ i for i in range(0,5)]

In [147]:
x

[0, 1, 2, 3, 4]

In [93]:
from xgboost import XGBClassifier
xgbm=XGBClassifier(objective="multi:softmax")
xgbm.fit(X_train,y_train)
y_pred=xgbm.predict(X_test)
acc=metrics.accuracy_score(y_test,y_pred) 
#here we are passing actual result and predicted result to find out accuracy
print('acuuracy>>>>',acc*100)


acuuracy>>>> 64.18578623391159
